In [1]:
import socket
import time
import struct
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def recv_n_bytes(s, n):
    buffer = bytearray()
    while len(buffer) < n:
        data = s.recv(n - len(buffer))
        if not data:
            if len(buffer) > 0:
                return buffer
            return None
        buffer.extend(data)
    return buffer

In [15]:
dps = []

In [16]:
host = "192.168.0.102"
port = 12345

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((host, port))
server.listen(1)

print(f"Listening on {host}:{port}...")

client_socket, client_address = server.accept()
print(f"Connection from {client_address}")

start = time.time()
while time.time() - start < 60:
    data = recv_n_bytes(client_socket, 24)
    dps.append(struct.unpack('ffffff', data))
    # for i in range(0,600,24):
    #     dps.append(struct.unpack('ffffff', data[i:i+24]))

client_socket.close()
server.close()

Listening on 192.168.0.102:12345...
Connection from ('192.168.0.150', 56527)


In [17]:
1000 / 10 * 60

6000.0

In [18]:
df = pd.DataFrame(dps, columns=['acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y', 'gry_z'])

In [19]:
df

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gry_z
0,-0.712280,0.466064,0.531250,0.000000,-0.183105,0.671387
1,-0.715576,0.467651,0.530396,0.244141,0.183105,0.000000
2,-0.717041,0.464111,0.527466,0.305176,0.427246,-0.122070
3,-0.718750,0.465820,0.527710,0.427246,0.610352,-0.183105
4,-0.721313,0.468262,0.528931,0.366211,0.671387,-0.183105
...,...,...,...,...,...,...
5529,-0.726807,0.467896,0.527832,0.122070,0.183105,0.305176
5530,-0.725830,0.468262,0.529297,0.183105,0.000000,0.610352
5531,-0.719116,0.466187,0.526367,0.122070,0.000000,0.732422
5532,-0.720215,0.466553,0.528076,0.061035,0.000000,0.793457


In [20]:
1/((4736 / 60) / 1000)

12.668918918918918

In [21]:
1/((5534 / 60) / 1000)

10.84206722081677

In [ ]:
def crt_to_plr(xyz):
    plr = np.zeros_like(xyz)
    xy = xyz[:,0]**2 + xyz[:,1]**2
    plr[:,0] = np.sqrt(xy + xyz[:,2]**2)
    plr[:,1] = np.arctan2(xyz[:,1], xyz[:,0])
    plr[:,2] = np.arctan2(np.sqrt(xy), xyz[:,2])
    return plr

In [ ]:
def plr_to_crt(rtp):
    crt = np.zeros_like(rtp)
    crt[:,0] = rtp[:, 0] * np.cos(rtp[:, 1]) * np.sin(rtp[:, 2])
    crt[:,1] = rtp[:, 0] * np.sin(rtp[:, 1]) * np.sin(rtp[:, 2])
    crt[:,2] = rtp[:, 0] * np.cos(rtp[:, 2])
    return crt

In [ ]:
def sub_grav(xyz):
    plr = crt_to_plr(xyz)
    plr[:,0] = plr[:,0] - 1.0
    return plr_to_crt(plr)

In [22]:
df = pd.read_csv('test_data1.csv')

In [ ]:
df[['adj_acc_x', 'adj_acc_y', 'adj_acc_z']] = sub_grav(df[['acc_x', 'acc_y', 'acc_z']].values)

In [ ]:
df.to_csv('./test_data2.csv')

In [ ]:
data = df[['adj_acc_x', 'adj_acc_y', 'adj_acc_z']].values

In [ ]:
vel = (data.cumsum(axis=-1) ** 2.0).sum(axis=-1) ** 0.2

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(0, len(vel) / 100, 0.01), vel)